In [20]:
import time

grid_size = 5
state=0
goal=4
total_reward=0

**Grids visualisation**

In [23]:
def show_grid(state):
  grid=""
  for i in range(grid_size):
    if i==state:
      grid +="P "
    elif i==goal:
      grid +="G "
    else:
      grid +="_ "
  print(grid)

print("Initial Environment")
show_grid(state)

Initial Environment
P _ _ _ G 


In [24]:
while state != goal:
  action = int(input("Enter action (0 = Left, 1 = Right): "))

  if action == 1 and state < goal:
    state += 1
  elif action == 0 and state > 0:
    state -= 1

  reward = -1
  if state == goal:
    reward = 10

  total_reward += reward

  show_grid(state)
  print("Reward: ", reward)
  time.sleep(0.5)

Enter action (0 = Left, 1 = Right): 0
P _ _ _ G 
Reward:  -1
Enter action (0 = Left, 1 = Right): 1
_ P _ _ G 
Reward:  -1
Enter action (0 = Left, 1 = Right): 1
_ _ P _ G 
Reward:  -1
Enter action (0 = Left, 1 = Right): 1
_ _ _ P G 
Reward:  -1
Enter action (0 = Left, 1 = Right): 1
_ _ _ _ P 
Reward:  10


Agent: P ,_ ,g
Environment:grid
Actions:left and right
Reward:-1,10